In [2]:
import findspark
findspark.init('/home/kant/spark-2.4.4-bin-hadoop2.7')

In [5]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName('Spam Detector').getOrCreate()

In [8]:
data = spark.read.csv('smsspamcollection/SMSSpamCollection', inferSchema=True, sep='\t')

In [11]:
data = data.withColumnRenamed('_c0',"class").withColumnRenamed('_c1',"text")

In [13]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [15]:
from pyspark.sql.functions import length

In [16]:
data = data.withColumn('length', length(data['text']))

In [17]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [18]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [19]:
from pyspark.ml.feature import Tokenizer,IDF,StringIndexer,CountVectorizer,StopWordsRemover

In [22]:
tokenizer = Tokenizer(inputCol='text',outputCol='token_text')
stop_remove = StopWordsRemover(inputCol='token_text',outputCol='stop_token')
count_vec = CountVectorizer(inputCol='stop_token', outputCol='count_vc')
idf = IDF(inputCol='count_vc',outputCol='tf_idf')
ham_spam_to_numeric = StringIndexer(inputCol='class',outputCol='label')

In [23]:
from pyspark.ml.feature import VectorAssembler

In [24]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'], outputCol='features')

In [25]:
from pyspark.ml.classification import NaiveBayes

In [26]:
nb = NaiveBayes()

In [28]:
from pyspark.ml import Pipeline

In [31]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_numeric,tokenizer,stop_remove,count_vec,idf,clean_up])

In [32]:
cleaner = data_prep_pipe.fit(data)

In [33]:
clean_data = cleaner.transform(data)

In [35]:
clean_data = clean_data.select('label','features')

In [36]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [37]:
training_data, test_data = clean_data.randomSplit([0.7,0.8], seed=33)

In [38]:
spam_detector = nb.fit(training_data)

In [39]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [41]:
test_result = spam_detector.transform(test_data)

In [42]:
test_result.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,41,...|[-1058.3364517980...|[1.0,7.7883177240...|       0.0|
|  0.0|(13424,[0,1,4,50,...|[-838.96445012744...|[1.0,4.2205313354...|       0.0|
|  0.0|(13424,[0,1,5,15,...|[-998.03934908390...|[1.0,1.0171353516...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-862.09651533698...|[1.0,1.1126160999...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-1148.1620069647...|[1.0,1.4001409103...|       0.0|
|  0.0|(13424,[0,1,7,15,...|[-685.02624465974...|[1.0,1.7961202671...|       0.0|
|  0.0|(13424,[0,1,11,32...|[-890.09479144850...|[1.0,1.0277494978...|       0.0|
|  0.0|(13424,[0,1,15,20...|[-668.38894928685...|[1.0,3.0567539013...|       0.0|
|  0.0|(13424,[0,1,20,27...|[-953.06546000875...|[1.0,3.9354180992...|       0.0|
|  0.0|(13424,[0